# How many max features?

In [1]:
import mlflow

# Set local MLflow tracking URI
mlflow.set_tracking_uri("file:./mlruns")

# Set experiment (will be created automatically if not exists)
mlflow.set_experiment("Exp 3 - BoW Bigram max_features")

/home/eemanasad2/venvs/mlops/lib/python3.12/site-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2025/12/23 14:59:32 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - BoW Bigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///home/eemanasad2/mlops/Youtube Sentiment '
 'Insights/notebooks/mlruns/518042296186650903'), creation_time=1766483972658, experiment_id='518042296186650903', last_update_time=1766483972658, lifecycle_stage='active', name='Exp 3 - BoW Bigram max_features', tags={}>

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [3]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [4]:
# Step 1: Function to run the experiment
def run_experiment_bow_max_features(max_features):
    ngram_range = (1, 2)  # bigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"BoW_Bigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"Random Forest with BoW Bigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: BoW Bigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_bow_bigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_bow_max_features(max_features)

2025/12/23 14:59:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 15:00:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 15:00:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 15:00:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 15:00:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 15:00:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 15:00:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 15:01:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 15:01:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 15:01:20 WARNING mlflow.mo